# Augment A Knowledge Graph Using CSV Files

A common knowledge graph augmentation use case is to integrate structured data present in databases, CSV and Excel files. KGTK provides an alternative to languages such as R2RML (https://www.w3.org/TR/r2rml/) and RML (https://rml.io/specs/rml/) or tools such as Karma (https://github.com/usc-isi-i2/Web-Karma) for integrating tabluar data. These languages and tools work by defining a mapping between a structured souce and an ontology. KGTK provides capabilities to transform the original structured data into the TSV format used in KGTK to store KGs. 

This tutorial illustrates the KGTK approach using a CSV file downloaded from Kaggle (https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset) containing information about moves from IMDb (https://www.imdb.com/).

This tutorial is divided into multiple sections:
- Survey what is available in our KG and in the IMDb file
- Prepare the IMDb file for ingestion in KGTK
- Find overlap betwwen our KG and the IMDb KG
- Integrate knowledge from the IMDb KG into our KG
- Implement a better model for the data

In [11]:
import io
import os
import subprocess
import sys

import numpy as np
import pandas as pd
from IPython.display import display, HTML
import csv

import papermill as pm

from kgtk.configure_kgtk_notebooks import ConfigureKGTK
from kgtk.functions import kgtk, kypher

In [12]:
# Parameters

#kgtk_path = "/Users/pedroszekely/Documents/GitHub/kgtk"

# Folder on local machine where to create the output and temporary folders
# input_path = "/Volumes/saggu-ssd/kgtk-tutorial-files/datasets/arnold-profiled"
input_path = None

output_path = "/tmp/projects"
project_name = "tutorial-augment"

These are all the KG files that we use in this tutorial:

In [13]:
files = [
    "all",
    "label",
    "alias",
    "description",
    "external_id",
    "monolingualtext",
    "quantity",
    "string",
    "time",
    "item",
    "wikibase_property",
    "qualifiers",
    "datatypes",
    "p279",
    "p279star",
    "p31",
    "in_degree",
    "out_degree",
    "pagerank_directed",
    "pagerank_undirected"
]

ck = ConfigureKGTK(files)
ck.configure_kgtk(input_graph_path=input_path,
                  output_path=output_path,
                  project_name=project_name)


User home: /Users/grantxie
Current dir: /Users/grantxie/Downloads/kgtk-notebooks/tutorial
KGTK dir: /Users/grantxie/Downloads/kgtk-notebooks
Use-cases dir: /Users/grantxie/Downloads/kgtk-notebooks/use-cases
--2022-02-01 09:49:37--  https://github.com/usc-isi-i2/kgtk-tutorial-files/raw/main/datasets/arnold/all.tsv.gz
Resolving github.com... 140.82.114.4
Connecting to github.com|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz [following]
--2022-02-01 09:49:38--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/arnold/all.tsv.gz
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/all.tsv.gz [following]
--2022-02-01 09:49:38--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/arnold/

In [14]:
ck.print_env_variables()

KGTK_GRAPH_CACHE: /tmp/projects/tutorial-augment/temp.tutorial-augment/wikidata.sqlite3.db
EXAMPLES_DIR: /Users/grantxie/Downloads/kgtk-notebooks/examples
kypher: kgtk query --graph-cache /tmp/projects/tutorial-augment/temp.tutorial-augment/wikidata.sqlite3.db
GRAPH: /Users/grantxie/isi-kgtk-tutorial/input
kgtk: kgtk
KGTK_OPTION_DEBUG: false
USE_CASES_DIR: /Users/grantxie/Downloads/kgtk-notebooks/use-cases
TEMP: /tmp/projects/tutorial-augment/temp.tutorial-augment
OUT: /tmp/projects/tutorial-augment
KGTK_LABEL_FILE: /Users/grantxie/isi-kgtk-tutorial/input/labels.en.tsv.gz
STORE: /tmp/projects/tutorial-augment/temp.tutorial-augment/wikidata.sqlite3.db
all: /Users/grantxie/isi-kgtk-tutorial/input/all.tsv.gz
label: /Users/grantxie/isi-kgtk-tutorial/input/labels.en.tsv.gz
alias: /Users/grantxie/isi-kgtk-tutorial/input/aliases.en.tsv.gz
description: /Users/grantxie/isi-kgtk-tutorial/input/descriptions.en.tsv.gz
external_id: /Users/grantxie/isi-kgtk-tutorial/input/claims.external-id.tsv.gz
m

Set up defaults KGTK

In [15]:
os.environ['KGTK_GRAPH_CACHE'] = os.environ['STORE']

Load all my files into the kypher cache so that all graph aliases are defined

In [16]:
%%time
ck.load_files_into_cache()

kgtk query --graph-cache /tmp/projects/tutorial-augment/temp.tutorial-augment/wikidata.sqlite3.db -i "/Users/grantxie/isi-kgtk-tutorial/input/all.tsv.gz" --as all  -i "/Users/grantxie/isi-kgtk-tutorial/input/labels.en.tsv.gz" --as label  -i "/Users/grantxie/isi-kgtk-tutorial/input/aliases.en.tsv.gz" --as alias  -i "/Users/grantxie/isi-kgtk-tutorial/input/descriptions.en.tsv.gz" --as description  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.external-id.tsv.gz" --as external_id  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.monolingualtext.tsv.gz" --as monolingualtext  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.quantity.tsv.gz" --as quantity  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.string.tsv.gz" --as string  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.time.tsv.gz" --as time  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.wikibase-item.tsv.gz" --as item  -i "/Users/grantxie/isi-kgtk-tutorial/input/claims.wikibase-property.tsv.gz" --as wikibase_pro

## Survey the data in the IMDb file

The first step is to determine whether our KG has IMDb identifiers, as this will make it easy to integrate the IMDb data.
The following query counts the number of items in our KG that have an `IMDb ID (P345)`.
Fortunately, our KG has many entities with IMDb identifiers:

In [7]:
kgtk("""
    query -i external_id
        --match '(movie)-[:P345]->(imdbid)'
        --return 'count(distinct movie) as `"movies with IMDb identifiers"`'
""")

,movies with IMDb identifiers
0,6698


As there are many items that have IMDb ids, we can augment our graph using data from IMDB. The following file comes from Kaggle: https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset

Our KG, a subset of Wikidata is missing many fields that are present in the IMDb file. 
For example, when we look at `film (Q11424)` in our browser http://ckg07.isi.edu:3008/browser/Q11424, we see that there is no information about ratings.

In [17]:
url = "https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/imdb"
other_files = [
    "IMDB.csv.gz",
    "imdb-kg-node.tsv",
    "imdb-kg.tsv"
]

for file in other_files:
    cmd = f" wget {url}/{file} --directory-prefix={os.environ['GRAPH']}/imdb"
    print(subprocess.getoutput(cmd))

--2022-02-01 09:54:01--  https://github.com/usc-isi-i2/kgtk-notebooks/raw/main/datasets/imdb/IMDB.csv.gz
Resolving github.com... 140.82.112.3
Connecting to github.com|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/imdb/IMDB.csv.gz [following]
--2022-02-01 09:54:03--  https://raw.githubusercontent.com/usc-isi-i2/kgtk-notebooks/main/datasets/imdb/IMDB.csv.gz
Resolving raw.githubusercontent.com... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21726321 (21M) [application/octet-stream]
Saving to: '/Users/grantxie/isi-kgtk-tutorial/input/imdb/IMDB.csv.gz'

     0K .......... .......... .......... .......... ..........  0% 2.63M 8s
    50K .......... .......... .......... .......... ..........  0% 4.67M 6s
   100K .........

In [19]:
imdb = pd.read_csv(os.environ['GRAPH'] + "/imdb/IMDB.csv.gz")
imdb

/var/folders/08/r2trmhq1723gfxg596vwcnw00000gn/T/ipykernel_21020/572855786.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv(os.environ['GRAPH'] + "/imdb/IMDB.csv.gz")


,imdb_title_id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


## Prepare the IMDb file for ingestion in KGTK
This step illustrates the power of the KGTK approach: you canb convert the CSV file into a KG by simply renaming the column in the file that contains the identifier for the records to have the column heading `id`. Once you do this, the file becomes a KGTK node file, where the `id` column identifies a node, and the other columns define properties about the node. 

The initial node file is only a first approaximation, as additional transformations will be needed to make the data compatible with the KG.
In this step, you will use use Pandas to convert the IMDb file to a KG node file by renaming the `imdb_title_id` column to `id` as it is the identifier for a movie.

We will also convert non-numeric values in the cells to strings to enable us later to use string transformations to clean the data.

In [20]:
imdb_kg = imdb.rename(columns={'imdb_title_id':'id'})
imdb_kg

,id,title,original_title,year,date_published,genre,duration,country,language,director,...,actors,description,avg_vote,votes,budget,usa_gross_income,worlwide_gross_income,metascore,reviews_from_users,reviews_from_critics
0,tt0000009,Miss Jerry,Miss Jerry,1894,1894-10-09,Romance,45,USA,None,Alexander Black,...,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,5.9,154,NaN,NaN,NaN,NaN,1.0,2.0
1,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,1906,1906-12-26,"Biography, Crime, Drama",70,Australia,None,Charles Tait,...,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,6.1,589,$ 2250,NaN,NaN,NaN,7.0,7.0
2,tt0001892,Den sorte drøm,Den sorte drøm,1911,1911-08-19,Drama,53,"Germany, Denmark",NaN,Urban Gad,...,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,5.8,188,NaN,NaN,NaN,NaN,5.0,2.0
3,tt0002101,Cleopatra,Cleopatra,1912,1912-11-13,"Drama, History",100,USA,English,Charles L. Gaskill,...,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,5.2,446,$ 45000,NaN,NaN,NaN,25.0,3.0
4,tt0002130,L'Inferno,L'Inferno,1911,1911-03-06,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",...,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,7.0,2237,NaN,NaN,NaN,NaN,31.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,tt9908390,Le lion,Le lion,2020,2020-01-29,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,...,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,5.3,398,NaN,NaN,$ 3507171,NaN,NaN,4.0
85851,tt9911196,De Beentjes van Sint-Hildegard,De Beentjes van Sint-Hildegard,2020,2020-02-13,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,...,"Herman Finkers, Johanna ter Steege, Leonie ter...",A middle-aged veterinary surgeon believes his ...,7.7,724,NaN,NaN,$ 7299062,NaN,6.0,4.0
85852,tt9911774,Padmavyuhathile Abhimanyu,Padmavyuhathile Abhimanyu,2019,2019-03-08,Drama,130,India,Malayalam,Vineesh Aaradya,...,"Anoop Chandran, Indrans, Sona Nair, Simon Brit...",NaN,7.9,265,NaN,NaN,NaN,NaN,NaN,NaN
85853,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,2019,2019-03-15,"Drama, Family",98,Turkey,Turkish,Ahmet Faik Akinci,...,"Ahmet Faik Akinci, Belma Mamati, Metin Keçeci,...",NaN,6.4,194,NaN,NaN,$ 2833,NaN,NaN,NaN


In KGTK, most commands rquire KGs to be in edge format (`node1/labal/node2`), so we use the `normalize-nodes` command to convert the IMDb nodes file to edge format.
The IMDb file contains 85K rows and 22 columns, gemerating 1.8 million edges.

The IMDb KG is completely isolated from our KG, but it is still a valid KG that we4 can manipulate in KGTK:
> In KGTK, literals, such as the string identifiers of movies, can be used as the subjects (`node1`) of triples.

After converting the original CSV file to a node file,  you can use the `normmalize-nodes` command to convert the node file to and edge file usinf the `node1/label/node2` headings. The columns of the original file become the labels of the edges and appear in the second column. Having done this, the original IMDB file has become a KG that you can transform using other KGTK commands.

In [21]:
%%time
kgtk(imdb_kg, """
    normalize-nodes
""")

CPU times: user 9.15 s, sys: 2.82 s, total: 12 s
Wall time: 11.9 s


,node1,label,node2
0,tt0000009,title,Miss Jerry
1,tt0000009,original_title,Miss Jerry
2,tt0000009,year,1894
3,tt0000009,date_published,1894-10-09
4,tt0000009,genre,Romance
...,...,...,...
1802950,tt9914942,usa_gross_income,NaN
1802951,tt9914942,worlwide_gross_income,$ 59794
1802952,tt9914942,metascore,NaN
1802953,tt9914942,reviews_from_users,NaN


Save the IMDb KG to a temorary file and give it the alias `imdbkg`:

In [22]:
%%time
kgtk(imdb_kg, """
    normalize-nodes -o $TEMP/imdb-kg-edge.tsv
""")

kgtk("query -i $TEMP/imdb-kg-edge.tsv --as imdbkg --limit 3")

CPU times: user 1.87 s, sys: 309 ms, total: 2.18 s
Wall time: 14.2 s


,node1,label,node2
0,tt0000009,title,Miss Jerry
1,tt0000009,original_title,Miss Jerry
2,tt0000009,year,1894


## Find overlap betwwen our KG and the IMDb KG

The IMDb KG uses IMDb identifiers as nodes, and our KG, extracted from Wikidata, defined property `IMDb ID (P345)` to record the IMDb identifiers of movies.

We can use KGTK to query both graphs simultaneously to find the movies in our graph for which there are nodes in the IMDb KG.
> We are only interested in the `node1` in the IMDb KG , so we don't have to list the other elements of the pattern, thee `label` and `node2`.

In [23]:
kgtk("""
    query -i imdbkg -i external_id
        --match '
            imdb: (imdb_id),
            external_id: (movie)-[:P345]->(imdb_id)
            '
        --return 'distinct movie as id, imdb_id as P345'
""")

,id,P345
0,Q1009788,tt0071360
1,Q1012216,tt0095243
2,Q102448,tt0304141
3,Q1025096,tt0094824
4,Q1026724,tt0046035
...,...,...
1286,Q977196,tt0098987
1287,Q978974,tt0104427
1288,Q980041,tt0983193
1289,Q980308,tt0080661


Count the number of entities in our KG that have an `IMDb ID (P345)`:

In [24]:
kgtk("""
    query -i external_id
        --match '(movie)-[:P345]->()'
        --return 'count(distinct movie) as count_movies'
""")

,count_movies
0,6698


Find the number of `film (Q11424)` in our KG.
> Our IMDb KG only has information for films.

In [25]:
kgtk("""
    query -i all
        --match '(film)-[:P31]->(class)-[:P279star]->(:Q11424)'
        --return 'count(distinct film) as count_film'
""")

,count_film
0,2447


So, over 6,000 entities in our KG have IMDb identifers, about half of these are films, and about half of those appear in our IMDb KG.

## Integrate knowledge from the IMDb KG into our KG

There are many fields in the IMDb KG that we could integrate into our KG. In this tutorial we will focus on three fields: the `reviews_from_users`, `reviews_from_critics` and `duration`.

Approach:
- Query both KGs, joining on the IMDb identifier, and extract the  properties we want
- ETL the data to conform to the Wikidata convetions used in our KG

Let's start with `reviews_from_users`, `reviews_from_critics`, which are counts. If we search in the browser we see very specific properties named `number of ...`, but none referring to number of reviews, so we will create two new properties, and we will give them the identifiers `Pnumber_of_user_reviews` and `Pnumber_of_critic_reviews`
> We follow the Wikidata convention where the identifiers for properties start with `P` and the identifiers for entities start with `Q`. KGTK does not require to use numbers following `P` or `Q`, and it does not require that we follow the `P/Q` convention.

To integrate the data we will use two KGs:
- The `imdbkg` with the data we pulled directly from the IMDb csv file
- The `external_id` subset of our KG that contains all external identifiers for all items in our KG.

The `match` clause does a join on the identifers, enabling us to pull from the `imdbkg` the items thatalso exist in our KG.
The `opt` clause (optional) retrieves the `reviews_from_users` property from the `imdbkg` KG
We use the `opt` clause so that if the `imdbkg` has a null, we still output the result.

The `return` clause builds a new node file: we use the `film` node from our KG as the `id` for our node file, and we translate the `reviews_from_users` data from the `imdbkg` as a `Pnumber_of_user_reviews`, a new property in our KG (below we extend this query to integrate all the data).
> In KGTK, you can integrate data from external sources using queries to map from one schema (ontology) to another, and do ETL transformations in one place.

Our KG is starting to take shape. For every film in the intersection of the two KGs, we now have a new property `Pnumber_of_user_reviews`:

In [26]:
kgtk("""
    query -i imdbkg -i external_id
        --match '
            external_id: (film)-[:P345]->(imdb_id),
            imdb: (imdb_id)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_users]->(rfu)'
        --return 'distinct
            film as id,  
            rfu as Pnumber_of_user_reviews
            '
        --order-by 'cast(rfu, int) desc'
""")

,id,Pnumber_of_user_reviews
0,Q172241,8232.0
1,Q163872,6938.0
2,Q18486021,6718.0
3,Q127367,5392.0
4,Q6074,4822.0
...,...,...
1286,Q51274461,2.0
1287,Q5437557,1.0
1288,Q21427084,NaN
1289,Q51274879,NaN


You can extend the query to integrate `reviews_from_critics` and `duration` by adding additional `opt` clauses.
Wikidata has propery `duration (P2047`), so we reuse it:

In [27]:
kgtk("""
    query -i imdbkg -i external_id
        --match '
            external_id: (film)-[:P345]->(imdb_id),
            imdb: (imdb_id)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_users]->(rfu)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_critics]->(rfc)'
        --opt '
            imdb: (imdb_id)-[:duration]->(duration)'
        --return 'distinct
            film as id,  
            rfu as Pnumber_of_user_reviews,
            rfc as Pnumber_of_critic_reviews,
            duration as P2047
            '
        --order-by 'cast(rfu, int) desc'
""")

,id,Pnumber_of_user_reviews,Pnumber_of_critic_reviews,P2047
0,Q172241,8232.0,164.0,142
1,Q163872,6938.0,423.0,152
2,Q18486021,6718.0,717.0,152
3,Q127367,5392.0,340.0,178
4,Q6074,4822.0,909.0,138
...,...,...,...,...
1286,Q51274461,2.0,8.0,78
1287,Q5437557,1.0,1.0,100
1288,Q21427084,NaN,6.0,90
1289,Q51274879,NaN,3.0,71


If we browse `Terminator 2: Judgment Day (Q170564)`, we see that we ought to define units of measure (`minute (Q7727)`).
In KGTK, quantities such as duration are represented as structured literals that incorporate the quantity and the units in one symbol.
For example "142 minutes" is represented as `142Q7727`.

It is easy to incorporate the units into our query by using the `printf` statement to combine multiple variables into a formatted string:
> The case of units of measure is a good example of the benefit to do schema mapping and ETL in the same query.

> We illustrate the use the KGTK function `kgtk_quantity_number` in the `order-by` clause to extract the numeric value of a quantity structured literals (https://kgtk.readthedocs.io/en/latest/transform/query/#functions-on-kgtk-numbers-and-quantities)

In [28]:
%%time
kgtk("""
    query -i imdbkg -i external_id
        --match '
            external_id: (film)-[:P345]->(imdb_id),
            imdb: (imdb_id)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_users]->(rfu)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_critics]->(rfc)'
        --opt '
            imdb: (imdb_id)-[:duration]->(duration)'
        --return 'distinct
            film as id,  
            rfu as Pnumber_of_user_reviews,
            rfc as Pnumber_of_critic_reviews,
            printf("+%sQ7727", duration) as P2047
            '
        --order-by 'kgtk_quantity_number(P2047) desc'
""")

CPU times: user 10.1 ms, sys: 15.4 ms, total: 25.4 ms
Wall time: 995 ms


,id,Pnumber_of_user_reviews,Pnumber_of_critic_reviews,P2047
0,Q1218601,127.0,22.0,+260Q7727
1,Q2875,881.0,197.0,+238Q7727
2,Q228186,676.0,146.0,+228Q7727
3,Q746733,293.0,76.0,+220Q7727
4,Q148204,233.0,137.0,+219Q7727
...,...,...,...,...
1286,Q51274879,NaN,3.0,+71Q7727
1287,Q5532208,14.0,9.0,+71Q7727
1288,Q736731,51.0,39.0,+71Q7727
1289,Q43051,181.0,118.0,+70Q7727


The query above extracts data from the simple `imdbkg` graph that uses the column headings as properties. The query maps the properties and data to new or existing Wikidata properties that we use in our KG, and performs simple data cleaning to conform to the Wikidata and KGTK format requirement. The resulting KG is in KGTK node format where the properties appear in the column headings; we save this graph in `$TEMP/imdb-import.node.tsv`:

> It is possible to create the KGTK node file using Pandas as the node file has very simmilar structure to the original CSV file. One of the advantages of KGTK is that it empowers users to use tools that they are familiar with, e.g., Pandas, to transform KGTK graphs. Integration with KGTK is seamless.

In [29]:
kgtk("""
    query -i imdbkg -i external_id
        --match '
            external_id: (film)-[:P345]->(imdb_id),
            imdb: (imdb_id)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_users]->(rfu)'
        --opt '
            imdb: (imdb_id)-[:reviews_from_critics]->(rfc)'
        --opt '
            imdb: (imdb_id)-[:duration]->(duration)'
        --return 'distinct
            film as id,  
            rfu as Pnumber_of_user_reviews,
            rfc as Pnumber_of_critic_reviews,
            printf("+%sQ7727", duration) as P2047
            '
    -o $TEMP/imdb-import.node.tsv
""")

The last step is to convert the node file to a an edge file so that it is in the standard form used by all KGTK commands.
Convert the node file to edges and add edge ids:

> If you had used Pandas to create the graph in node format, this same command would transform it into and edge file, and make the4 graph ready for use with any other KGTL commands.

In [30]:
kgtk("""
    normalize-nodes -i $TEMP/imdb-import.node.tsv
    / add-id --id-style wikidata
""")

,node1,label,node2,id
0,Q1009788,Pnumber_of_user_reviews,386.0,Q1009788-Pnumber_of_user_reviews-7e80e4
1,Q1009788,Pnumber_of_critic_reviews,169.0,Q1009788-Pnumber_of_critic_reviews-ff2927
2,Q1009788,P2047,+113Q7727,Q1009788-P2047-54edad
3,Q1012216,Pnumber_of_user_reviews,69.0,Q1012216-Pnumber_of_user_reviews-d411e0
4,Q1012216,Pnumber_of_critic_reviews,28.0,Q1012216-Pnumber_of_critic_reviews-3884d7
...,...,...,...,...
3868,Q980308,Pnumber_of_critic_reviews,149.0,Q980308-Pnumber_of_critic_reviews-c70c59
3869,Q980308,P2047,+104Q7727,Q980308-P2047-75da46
3870,Q997206,Pnumber_of_user_reviews,69.0,Q997206-Pnumber_of_user_reviews-d411e0
3871,Q997206,Pnumber_of_critic_reviews,36.0,Q997206-Pnumber_of_critic_reviews-f85f0b


Store the IMDb data, now represented in the Wikidata ontology, into a KG file and give it alias `augment_imdb` so that we can use it in the next steps:

In [31]:
kgtk("""
    normalize-nodes -i $TEMP/imdb-import.node.tsv
    / add-id --id-style wikidata
    -o $TEMP/augment.imdb.tsv
""")

kgtk("query -i $TEMP/augment.imdb.tsv --as augment_imdb --limit 2")

,node1,label,node2,id
0,Q1009788,Pnumber_of_user_reviews,386.0,Q1009788-Pnumber_of_user_reviews-7e80e4
1,Q1009788,Pnumber_of_critic_reviews,169.0,Q1009788-Pnumber_of_critic_reviews-ff2927


## Deduplicate the durations (`duration (P2047)`)
Wikidata defines durations for many films. If we want to integrate the IMDb data into our KG, we must be careful as there may be inconsitent durations.

Approach:
- Find the films where the durations in Wikidata and IMDB differ
- In case of conflict, prefer the IMDb durations

First, find films where the durations are different using a query that combines the orignal `all` graph and the `augment_imdb` graph.
There are many differences, and we see cases where the data in Wikidata is suspect (the uncertainty is the same as the value):

In [32]:
kgtk("""
    query -i all -i augment_imdb
        --match '
            all: (film)-[:P2047]->(duration_wd),
            augment_imdb: (film)-[:P2047]->(duration_imdb)
            '
        --where 'duration_wd < duration_imdb'
        --return 'distinct film as film, duration_wd as duration_wd, duration_imdb as duration_imdb'
        --order-by 'film'
    / add-labels
""")

,film,duration_wd,duration_imdb,film;label
0,Q1009788,+109Q7727,+113Q7727,'The Conversation'@en
1,Q102448,+136Q7727,+142Q7727,'Harry Potter and the Prisoner of Azkaban'@en
2,Q1025096,+93Q7727,+98Q7727,'Caddyshack II'@en
3,Q1027212,"+125[+125,+125]Q7727",+126Q7727,'Revolution'@en
4,Q103474,+143Q7727,+149Q7727,'2001: A Space Odyssey'@en
...,...,...,...,...
375,Q926825,"+128[+128,+128]Q7727",+133Q7727,'Flower Drum Song'@en
376,Q929647,+115Q7727,+120Q7727,'Mask'@en
377,Q930372,+175Q7727,+179Q7727,'Camelot'@en
378,Q936425,+128Q7727,+133Q7727,'A Very Long Engagement'@en


We decide to remove from our original graph the edges for durations that differ from the durations in IMDb. 
We enhance our query to return the ids of such edges:

In [33]:
kgtk("""
    query -i all -i augment_imdb
        --match '
            all: (film)-[l:P2047]->(duration_wd),
            augment_imdb: (film)-[:P2047]->(duration_imdb)
            '
        --where 'duration_wd < duration_imdb'
        --return 'distinct l as id'
        --order-by 'l'
""")

,id
0,Q1009788-P2047-16e299-ef37b7c7-0
1,Q102448-P2047-031b29-0e3d9b06-0
2,Q1025096-P2047-01387b-bea42f5f-0
3,Q1027212-P2047-c18970-a8e52174-0
4,Q103474-P2047-83a7f0-c5824aa3-0
...,...
375,Q926825-P2047-7abb58-87a8eb10-0
376,Q929647-P2047-b97fa8-669b35f9-0
377,Q930372-P2047-ed974c-06f071f1-0
378,Q936425-P2047-37faaa-7eb616cb-0


Store the ids of the discrepant duration edges in a file:

In [34]:
kgtk("""
    query -i all -i augment_imdb
        --match '
            all: (film)-[l:P2047]->(duration_wd),
            augment_imdb: (film)-[:P2047]->(duration_imdb)
            '
        --where 'duration_wd < duration_imdb'
        --return 'distinct l as id'
        --order-by 'l'
    -o $TEMP/id.discrepant_durations.tsv
""")

Compute the intersection and difference of the two files: the original `$quantity` file that contains all the quantity edges for our subset of Wikidata, and `$TEMP/id.discrepant_durations.tsv`, the file that contains the ids of duration edges in `$quantity` where the values in Wikidata and IMDb differ.

Use the KGTK `ifexists` command (https://kgtk.readthedocs.io/en/latest/transform/ifexists/).
Provide the two files, the names of the columns used as keys  (`id`), and tell the command to output the `reject-file` containing edges in the input file that are not present in the filter file (this is the file we want):

In [35]:
kgtk("""
    ifexists -i $quantity 
        --filter-on $TEMP/id.discrepant_durations.tsv
        --input-keys id
        --reject-file $OUT/quantity.minus_bad_durations.tsv
""")

,node1,label,node2,id,node2;wikidatatype
0,Q1009788,P2047,+109Q7727,Q1009788-P2047-16e299-ef37b7c7-0,quantity
1,Q102448,P2047,+136Q7727,Q102448-P2047-031b29-0e3d9b06-0,quantity
2,Q1025096,P2047,+93Q7727,Q1025096-P2047-01387b-bea42f5f-0,quantity
3,Q1027212,P2047,"+125[+125,+125]Q7727",Q1027212-P2047-c18970-a8e52174-0,quantity
4,Q103474,P2047,+143Q7727,Q103474-P2047-83a7f0-c5824aa3-0,quantity
...,...,...,...,...,...
375,Q926825,P2047,"+128[+128,+128]Q7727",Q926825-P2047-7abb58-87a8eb10-0,quantity
376,Q929647,P2047,+115Q7727,Q929647-P2047-b97fa8-669b35f9-0,quantity
377,Q930372,P2047,+175Q7727,Q930372-P2047-ed974c-06f071f1-0,quantity
378,Q936425,P2047,+128Q7727,Q936425-P2047-37faaa-7eb616cb-0,quantity


Sanity check: subtract the number of edges in the original quantity file `$quantity` minus the number of edges in `$OUT/quantity.minus_bad_durations.tsv`:

In [36]:
old = !zcat < $quantity | wc -l
new = !cat $OUT/quantity.minus_bad_durations.tsv | wc -l

int(old[0]) - int(new[0])

380

## Implement a better model for the IMDb data

The data model we just implemented is very simple. In this section, we will improve it to be more compatible with how Wikidata models data.
We will:
- Add a `point in time (P585)` qualifier to record the time when the reviews were counted. We will use 2020-01-01, the date reported in the Kaggle site
- Combine `Pnumber_of_user_reviews` and `Pnumber_of_critic_reviews` into a single `Pnumber_of_reviews` property, and use the `of (P642)` qualifier property 
to record whether the review counts are from a `customer (Q852835)` or a `critic (Q6430706)`

The approach we will use is to tranform the `augment_imdb` data we created above to have the desired structure. 
> An alternative approach would be to generate the data in the desired structure in the first place.

The following query transforms the `Pnumber_of_user_reviews` data into the desired structure. The work is done in the `return` statement:

In [37]:
kgtk("""
    query -i augment_imdb
        --match '(film)-[:Pnumber_of_user_reviews]->(count)'
        --return '
            film as node1,
            "Pnumber_of_reviews" as label,
            count as node2,
            "Q852835" as P642,
            "^2020-01-01T00:00:00Z/11" as P585
            '
""")

,node1,label,node2,P642,P585
0,Q1009788,Pnumber_of_reviews,386.0,Q852835,^2020-01-01T00:00:00Z/11
1,Q1012216,Pnumber_of_reviews,69.0,Q852835,^2020-01-01T00:00:00Z/11
2,Q102448,Pnumber_of_reviews,1600.0,Q852835,^2020-01-01T00:00:00Z/11
3,Q1025096,Pnumber_of_reviews,89.0,Q852835,^2020-01-01T00:00:00Z/11
4,Q1026724,Pnumber_of_reviews,20.0,Q852835,^2020-01-01T00:00:00Z/11
...,...,...,...,...,...
1286,Q977196,Pnumber_of_reviews,120.0,Q852835,^2020-01-01T00:00:00Z/11
1287,Q978974,Pnumber_of_reviews,74.0,Q852835,^2020-01-01T00:00:00Z/11
1288,Q980041,Pnumber_of_reviews,499.0,Q852835,^2020-01-01T00:00:00Z/11
1289,Q980308,Pnumber_of_reviews,233.0,Q852835,^2020-01-01T00:00:00Z/11


As always, we want an edge file with ids, so we need to add the requisite `add-id` and `normalize` commands:
> When doing this for real, we would not define multiple cells to revise the query. We would edit the cell, refining the pipeline to produce the desired results:

In [38]:
kgtk("""
    query -i augment_imdb
        --match '(film)-[:Pnumber_of_user_reviews]->(count)'
        --return '
            film as node1,
            "Pnumber_of_reviews" as label,
            count as node2,
            "Q852835" as P642,
            "^2020-01-01T00:00:00Z/11" as P585
            '
    / add-id --id-style wikidata
    / normalize --add-id True
""")

,node1,label,node2,id
0,Q1009788,Pnumber_of_reviews,386.0,Q1009788-Pnumber_of_reviews-7e80e4
1,Q1009788-Pnumber_of_reviews-7e80e4,P642,Q852835,Q1009788-Pnumber_of_reviews-7e80e4-P642-Q85283...
2,Q1009788-Pnumber_of_reviews-7e80e4,P585,^2020-01-01T00:00:00Z/11,Q1009788-Pnumber_of_reviews-7e80e4-P585-^2020-...
3,Q1012216,Pnumber_of_reviews,69.0,Q1012216-Pnumber_of_reviews-d411e0
4,Q1012216-Pnumber_of_reviews-d411e0,P642,Q852835,Q1012216-Pnumber_of_reviews-d411e0-P642-Q85283...
...,...,...,...,...
3868,Q980308-Pnumber_of_reviews-ae91a5,P642,Q852835,Q980308-Pnumber_of_reviews-ae91a5-P642-Q852835...
3869,Q980308-Pnumber_of_reviews-ae91a5,P585,^2020-01-01T00:00:00Z/11,Q980308-Pnumber_of_reviews-ae91a5-P585-^2020-0...
3870,Q997206,Pnumber_of_reviews,69.0,Q997206-Pnumber_of_reviews-d411e0
3871,Q997206-Pnumber_of_reviews-d411e0,P642,Q852835,Q997206-Pnumber_of_reviews-d411e0-P642-Q852835...


Do, the same transformation for the counts of critic reviews, and concatenate the result for user (customaer) and critic into on result file `$TEMP/reviews.tsv`:

In [39]:
kgtk("""
    query -i augment_imdb
        --match '(film)-[:Pnumber_of_user_reviews]->(count)'
        --return '
            film as node1,
            "Pnumber_of_reviews" as label,
            count as node2,
            "Q852835" as P642,
            "^2020-01-01T00:00:00Z/11" as P585
            '
    / add-id --id-style wikidata
    / normalize --add-id True
    -o $TEMP/reviews.user.tsv
""")

kgtk("""
    query -i augment_imdb
        --match '(film)-[:Pnumber_of_critic_reviews]->(count)'
        --return '
            film as node1,
            "Pnumber_of_reviews" as label,
            count as node2,
            "Q6430706" as P642,
            "^2020-01-01T00:00:00Z/11" as P585
            '
    / add-id --id-style wikidata
    / normalize --add-id True
    -o $TEMP/reviews.critic.tsv
""")

kgtk("""
    cat -i $TEMP/reviews.user.tsv -i $TEMP/reviews.critic.tsv
    -o $TEMP/reviews.tsv
""")

Now we need to remove the edges using the old `Pnumber_of_user_reviews` and `Pnumber_of_critic_reviews` and put in the new ones. We will use the `filter` command to do "grep" on the old file, and return the lines that don't match `--invert True`. The `filter`command uses a simple pattern language of the form `node1 pattern ; lable pattern ; node2 pattern` (https://kgtk.readthedocs.io/en/latest/transform/filter/):

In [40]:
kgtk("""
    filter -i $TEMP/augment.imdb.tsv 
        --pattern '; Pnumber_of_user_reviews, Pnumber_of_critic_reviews ;'
        --invert True
""")

,node1,label,node2,id
0,Q1009788,P2047,+113Q7727,Q1009788-P2047-54edad
1,Q1012216,P2047,+129Q7727,Q1012216-P2047-321f7d
2,Q102448,P2047,+142Q7727,Q102448-P2047-f4e24d
3,Q1025096,P2047,+98Q7727,Q1025096-P2047-6323a5
4,Q1026724,P2047,+79Q7727,Q1026724-P2047-0eb45e
...,...,...,...,...
1286,Q977196,P2047,+104Q7727,Q977196-P2047-75da46
1287,Q978974,P2047,+140Q7727,Q978974-P2047-b00fbd
1288,Q980041,P2047,+107Q7727,Q980041-P2047-dec492
1289,Q980308,P2047,+104Q7727,Q980308-P2047-75da46


Here is the unix-like grep/cat pipeline that replaces the old statements with the new ones:

In [41]:
kgtk("""
    filter -i $TEMP/augment.imdb.tsv 
        --pattern '; Pnumber_of_user_reviews, Pnumber_of_critic_reviews ;'
        --invert True
    
    / cat -i - -i $TEMP/reviews.tsv
        -o $OUT/augment.imdb.tsv
""")

Let's double check the results. The data looks good.

To add the new data to our KG, and view it in the browser, we have to define the new `Pnumber_of_reviews` property, give it a label, aliases and description, and most importantly, define its `datatype` as `quantity`
> We will not do this step in this tutorial. See https://github.com/usc-isi-i2/kgtk/blob/dev/kgtk-properties/kgtk.properties.tsv for examples of how to define new properties.

In [42]:
kgtk("cat -i $OUT/augment.imdb.tsv")

,node1,label,node2,id
0,Q1009788,P2047,+113Q7727,Q1009788-P2047-54edad
1,Q1012216,P2047,+129Q7727,Q1012216-P2047-321f7d
2,Q102448,P2047,+142Q7727,Q102448-P2047-f4e24d
3,Q1025096,P2047,+98Q7727,Q1025096-P2047-6323a5
4,Q1026724,P2047,+79Q7727,Q1026724-P2047-0eb45e
...,...,...,...,...
9032,Q980308-Pnumber_of_reviews-c70c59,P642,Q6430706,Q980308-Pnumber_of_reviews-c70c59-P642-Q643070...
9033,Q980308-Pnumber_of_reviews-c70c59,P585,^2020-01-01T00:00:00Z/11,Q980308-Pnumber_of_reviews-c70c59-P585-^2020-0...
9034,Q997206,Pnumber_of_reviews,36.0,Q997206-Pnumber_of_reviews-f85f0b
9035,Q997206-Pnumber_of_reviews-f85f0b,P642,Q6430706,Q997206-Pnumber_of_reviews-f85f0b-P642-Q643070...


## Summary of this tutorial:

In this tutorial we:
- Extracted data from a CSV file to add to our KG
- Converted a CSV file into a KGTK graph using Pandas to create a simple KG (this step is similar to doing a direct mapping using R2RML https://www.w3.org/TR/rdb-direct-mapping/)
- Used KGTK to extract/transform/load (ETL) the data into a simple model consistent with the Wikidata ontology
- Transformed the simple model into a better model
- The output of this section is
  - `$OUT/augment.imdb.tsv`, containing the new edges we created from the IMDb file
  - `$OUT/quantity.minus_bad_durations.tsv`, a modification of the orignal `$quantity` file where we removed discrepant durations.

> The original IMDb file contains many other interesting fields that would be fun to import. Many of them such as genre, director, actors require much more work as we must link the names to entities in Wikidata. Entity linking is outside the scope of KGTK; tools such as OpenRefine (https://openrefine.org/) or our own Table Linker (https://github.com/usc-isi-i2/table-linker) can be used to do entity linking. The KGTK `replace-nodes` command is helpful to process files containing probabilistic "same-as" statements to integrate the results of external entity linking tools.

# Deploy the results (Optional)

Deploy the tutorial files after completing this notebook.

List all the files:

!ls -l {project_deployment_path}